# Fine-tuning LLaMA with QLoRA

This notebook shows steps to fine-tune a LLaMA family model using QLoRA / LoRA. Use Colab with a GPU.

### 1) Install libraries

In [ ]:
# !pip install transformers peft accelerate bitsandbytes datasets sentencepiece

### 2) Load model in 4-bit mode (example uses Llama-2 7B HF)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer
from peft import LoraConfig, get_peft_model
import torch

model_name = 'meta-llama/Llama-2-7b-hf'  # Ensure you have access / appropriate weights
tokenizer = AutoTokenizer.from_pretrained(model_name, use_fast=False)
model = AutoModelForCausalLM.from_pretrained(model_name, device_map='auto', load_in_4bit=True)
print('Model loaded (4-bit)')

### 3) Prepare a tiny instruction dataset

In [ ]:
from datasets import Dataset

data = {
    'instruction': ['Explain AI to a 10-year-old.', 'Define machine learning in one sentence.'],
    'response': ['AI is like a smart helper that learns from examples.', 'Machine learning is a way computers learn patterns from data.']
}
ds = Dataset.from_dict(data)

def format_example(ex):
    return {'text': f"### Instruction:\n{ex['instruction']}\n\n### Response:\n{ex['response']}"}

ds = ds.map(format_example)
print(ds[0])


### 4) Apply LoRA (PEFT)

In [ ]:
from peft import LoraConfig, get_peft_model
config = LoraConfig(r=64, lora_alpha=16, target_modules=['q_proj','v_proj'], lora_dropout=0.1, bias='none')
model = get_peft_model(model, config)
model.print_trainable_parameters()


### 5) Training with Trainer (example)

In [ ]:
from transformers import TrainingArguments, Trainer

def tokenize_fn(examples):
    return tokenizer(examples['text'], truncation=True, padding='max_length', max_length=512)

train_ds = ds.map(lambda x: tokenize_fn(x), batched=True)
train_ds.set_format(type='torch', columns=['input_ids','attention_mask'])

args = TrainingArguments(output_dir='./llama_qlora_out', per_device_train_batch_size=1, gradient_accumulation_steps=4, max_steps=200, learning_rate=2e-4, logging_steps=10)
trainer = Trainer(model=model, args=args, train_dataset=train_ds)
# trainer.train()  # Uncomment to run training
print('Trainer created. Uncomment trainer.train() to start training.')